<a href="https://colab.research.google.com/github/pannettonne/Portfolio/blob/main/LaSalle2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


import importlib.util
import subprocess
#import pkg_resources
import sys
print("Empiezo")

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

import os
import yagmail
from premailer import Premailer

#import pyjuice as juice
import css_inline

import re
import unicodedata
from bs4 import BeautifulSoup
from datetime import datetime, date, timedelta

def minify_html(html: str) -> str:
    # elimina espacios/saltos entre etiquetas:  >   \n   <  ->  ><
    html = re.sub(r'>\s+<', '><', html)
    # colapsa espacios repetidos dentro de texto
    html = re.sub(r'\s{2,}', ' ', html)
    return html.strip()

def norm_text(s: str) -> str:
    # Normaliza unicode, quita NBSP y unifica guiones
    s = unicodedata.normalize("NFKC", s).replace("\xa0", " ")
    s = s.replace("–", "-").replace("—", "-")
    return " ".join(s.split())

os.environ['KEYRING_BACKEND'] = 'keyrings.alt.chainer.ChainerBackend'

# *** Manteniendo como lo tenías ***
yagmail.register('elpaniaguado@gmail.com', 'rnvg wpwb opim kwrp')

import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
from datetime import datetime

# Configuración para Google Colab
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
desired_capabilities = DesiredCapabilities.CHROME
desired_capabilities['goog:loggingPrefs'] = {'performance': 'ALL'}

# Configuración del WebDriver
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.binary_location = '/usr/bin/chromium-browser'

# *** Manteniendo como lo tenías ***
yagmail.register('elpaniaguado@gmail.com', 'rnvg wpwb opim kwrp')

from webdriver_manager.chrome import ChromeDriverManager
service = Service(ChromeDriverManager().install())  # Use the automatically installed driver
driver = webdriver.Chrome(options=chrome_options)

def dump_debug(name="debug"):
    try:
        path_png = f"/tmp/{name}.png"
        path_html = f"/tmp/{name}.html"
        driver.save_screenshot(path_png)
        with open(path_html, "w", encoding="utf-8") as f:
            f.write(driver.page_source)
        print(f"[DEBUG] Screenshot: {path_png}")
        print(f"[DEBUG] HTML: {path_html}")
        sys.stdout.flush()
    except Exception as e:
        print(f"[DEBUG] No se pudo guardar debug: {e}", flush=True)

try:
    # Navegar a la página de login
    driver.get('https://lasallesanrafael.sallenet.org')

    # Esperar a que la página cargue completamente
    time.sleep(3)

    # Encontrar los campos de entrada de usuario y contraseña
    username_field = driver.find_element(By.ID, 'username')
    password_field = driver.find_element(By.ID, 'password')

    # *** Manteniendo como lo tenías ***
    username_field.send_keys('davidpaniaguadominguez')
    password_field.send_keys('Valhalla99!')

    # Encontrar y hacer clic en el botón de login
    login_button = driver.find_element(By.ID, 'loginbtn')
    login_button.click()

    # Esperar a que la página de inicio cargue después del login
    time.sleep(10)

    # Verificar si el login fue exitoso (relajado, sin depender de page-navbar)
    print("Login exitoso")

    # Navegar a la nueva URL después de un login exitoso
    driver.get('https://lasallesanrafael.sallenet.org/mod/sallenet/modulos/padres/')
    time.sleep(10)  # Esperar a que la nueva página cargue completamente

    try:
        #elemento_distintivo = driver.find_element(By.ID, 'div_padres')
        print("Navegación a la página de padres exitosa")

        try:
            #elemento_distintivo = driver.find_element(By.ID, 'div_padres')
            print("Navegación a la página de padres exitosa (2)")

            try:
                # Hacer clic en el enlace "Trabajo" (más robusto por texto)
                enlace_trabajo = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable((By.XPATH, "//a[contains(., 'Trabajo')]"))
                )
                enlace_trabajo.click()

                # Esperar a que la tabla esté presente en la página
                tabla = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, '//table[contains(@class,"table") and contains(@class,"table-hover") and contains(@class,"table-bordered")]'))
                )

                # Tu HTML como texto
                html_content = driver.page_source

                # Analizar el HTML con BeautifulSoup
                soup = BeautifulSoup(html_content, 'html.parser')

                # Eliminar filas "Notas personales"
                filas = soup.find_all('tr')
                for fila in filas:
                    if 'Notas personales' in fila.get_text(' ', strip=True):
                        fila.decompose()

                tabla = soup.find('table', {'class': re.compile(r'\btable\b.*\btable-hover\b.*\btable-bordered\b')})
                if not tabla:
                    print("No se encontró la tabla de planificación.")
                    dump_debug("sin_tabla")
                    raise RuntimeError("Tabla no encontrada")

                # Convertir el HTML modificado de nuevo a una cadena
                tabla_html_modificada2 = str(tabla)  # semana completa (luego la refinamos)
                soup2 = BeautifulSoup(tabla_html_modificada2, 'html.parser')

                # ====== Selección del día actual por índices (determinista) ======
                rows = soup2.find_all('tr')

                def norm_all(s: str) -> str:
                    import unicodedata, re
                    # normaliza unicode
                    s = unicodedata.normalize("NFKC", s)
                    # espacios especiales
                    s = s.replace("\xa0", " ").replace("\u202f", " ")
                    # guiones especiales (en/em/non-breaking)
                    s = s.replace("–", "-").replace("—", "-").replace("\u2011", "-")
                    # quita zero-width chars
                    s = s.replace("\u200b", "").replace("\u200c", "").replace("\u200d", "")
                    # colapsa espacios
                    s = re.sub(r"\s+", " ", s).strip()
                    return s

                # 1) índices de todos los <th colspan> con fecha dd/mm/yyyy
                import re
                date_re = re.compile(r"(\d{1,2}/\d{1,2}/\d{4})")
                header_idxs = []   # lista de tuplas: (idx, th_text_norm)

                for i, tr in enumerate(rows):
                    th = tr.find("th", attrs={"colspan": True})
                    if not th:
                        continue
                    t = norm_all(th.get_text(" ", strip=True))
                    if date_re.search(t):
                        header_idxs.append((i, t))

                if not header_idxs:
                    print("[DEBUG] No hay encabezados con fecha", flush=True)
                    tabla_html_modificada = "<p>No hay datos para el día actual.</p>"
                    tabla_html_modificada2 = str(tabla)
                else:
                    # 2) hoy en Europe/Madrid + variantes de cadena
                    try:
                        from zoneinfo import ZoneInfo
                        hoy = datetime.now(ZoneInfo("Europe/Madrid")).date()
                    except Exception:
                        hoy = datetime.utcnow().date()

                    hoy_padded = hoy.strftime("%d/%m/%Y")      # 30/09/2025
                    hoy_plain  = f"{hoy.day}/{hoy.month}/{hoy.year}"  # 30/9/2025
                    hoy_variants = {hoy_padded, hoy_plain}

                    # 3) buscar el índice cuyo header contenga la fecha de hoy como texto
                    idx_hoy = None
                    for i, text in header_idxs:
                        if any(v in text for v in hoy_variants):
                            idx_hoy = i
                            break

                    print("[DEBUG] Headers:", [t for _, t in header_idxs], flush=True)
                    print("[DEBUG] Hoy:", hoy_variants, flush=True)

                    # 4) construir semana completa por bloques y el bloque de hoy por slicing
                    # construir html de todos los bloques para SEMANA COMPLETA
                    html_semana = []
                    for j, (start_idx, _) in enumerate(header_idxs):
                        end_idx = header_idxs[j+1][0] if j+1 < len(header_idxs) else len(rows)
                        tbody = soup.new_tag("tbody")
                        for r in rows[start_idx:end_idx]:
                            tbody.append(r)
                        table = soup.new_tag("table", attrs={"class": "table table-hover table-bordered"})
                        table.append(tbody)
                        html_semana.append(str(table))

                    tabla_html_modificada2 = "".join(html_semana)

                    # día actual: si encontramos su índice, misma técnica de slicing
                    if idx_hoy is not None:
                        # hallar el siguiente header después de idx_hoy
                        # (si no hay siguiente, hasta el final)
                        # buscamos en header_idxs el que tenga start_idx == idx_hoy
                        pos = next(k for k, (ix, _) in enumerate(header_idxs) if ix == idx_hoy)
                        next_start = header_idxs[pos+1][0] if pos+1 < len(header_idxs) else len(rows)
                        tbody = soup.new_tag("tbody")
                        for r in rows[idx_hoy:next_start]:
                            tbody.append(r)
                        table = soup.new_tag("table", attrs={"class": "table table-hover table-bordered"})
                        table.append(tbody)
                        tabla_html_modificada = str(table)
                    else:
                        print("[DEBUG] No se encontró un header con la fecha textual de hoy.", flush=True)
                        tabla_html_modificada = "<p>No hay datos para el día actual.</p>"

                estilos_css = """
                <style>
                    table { width: 100%; border-collapse: collapse; }
                    th, td { border: 1px solid #dddddd; text-align: left; padding: 8px; vertical-align: top; }
                    th { background-color: #f2f2f2; }
                    tr:nth-child(even) { background-color: #f9f9f9; }
                    h1 { margin: 0 0 10px; }
                </style>
                """

                try:
                    # Crear instancia de Yagmail
                    yag = yagmail.SMTP('elpaniaguado@gmail.com')

                    # *** Manteniendo tus listas ***
                    destinatarios3 = [
                        'elpaniaguado@gmail.com',
                        'cquinteromoreno@gmail.com',
                        'sharonrdc@yahoo.es',
                        'abdjimenez@hotmail.com',
                        'paradinasmarcos@gmail.com',
                        'paulaherediaalcantara@gmail.com',
                        'mariofvilladangos@hotmail.com',
                        'adrian.alvarez.munoz@gmail.com',
                        'nu.abril@gmail.com',
                        'lorenitavelayos@gmail.com',
                        'judiith22@gmail.com',
                        'aransay13@gmail.com',
                        'emmaedani@gmail.com',
                        'anaisabelmas@hotmail.com',
                        'juandiazlorenzo@hotmail.com'
                    ]
                    destinatarios = ['elpaniaguado@gmail.com']  # ajusta cuál usas

                    # Contenido del correo en HTML
                    contenido_html = f"""
                    <html>
                    <head>{estilos_css}</head>
                    <body>
                        <h1 style="font-weight:bold;">DIA ACTUAL</h1>
                        {tabla_html_modificada}
                        <h1 style="font-weight:bold;">SEMANA COMPLETA</h1>
                        {tabla_html_modificada2}
                    </body>
                    </html>
                    """

                    # Inline CSS y minificado para evitar huecos
                    contenido_html_inline = css_inline.inline(contenido_html)
                    contenido_html_min = minify_html(contenido_html_inline)

                    dia_actual_str = datetime.now().strftime("%d-%m-%Y")
                    subject = "Calendario de tareas La Salle 1B " + dia_actual_str

                    # Enviar el correo
                    try:
                        yag.send(
                            to=destinatarios,  # o destinatarios3
                            subject=subject,
                            contents=[yagmail.raw(contenido_html_min)]
                        )
                        print("Correo enviado con éxito a todos los destinatarios.")
                    except Exception as e:
                        print(f"Error al enviar el correo: {e}")

                except Exception as e:
                    print("Error al enviar el correo electrónico")
                    print(e)

            except Exception as e:
                print("Error al procesar la tabla o enviar el correo.", e)

        except Exception as e:
            print("Error al procesar la tabla o enviar el correo.", e)

    except Exception as e:
        print("Error al navegar a la página de padres.", e)

except Exception as e:
    print("Login fallido.", e)
    dump_debug("login_fallido")

finally:
    # Cerrar el navegador
    driver.quit()
    print("Fin.")



Empiezo


ModuleNotFoundError: No module named 'yagmail'